In [1]:
import pandas as pd
import numpy as np
import pickle

In [2]:
#url = 'https://raw.githubusercontent.com/MariaRosendal/Enhancing-Price-Momentum-with-RNN/main/'
url= ""

## 1. Importing input and output variables

In [60]:
# Import X 
X_train = pickle.load(open('X_train_scaled.pkl', 'rb' ))
X_test = pickle.load(open('X_test_scaled.pkl', 'rb' ))

In [460]:
# Checking for NaN values
print(np.count_nonzero(np.isnan(X_train)), np.count_nonzero(np.isnan(X_test)))

0 0


In [461]:
X_train.shape, X_test.shape, 

((270000, 12, 9), (270000, 12, 9))

In [462]:
# Importing y
y = pd.read_csv(url+'y_reg.csv')
y['date'] =  pd.to_datetime(y['date'])
y = y.set_index(['date'])

In [465]:
# Defining length of y_train and y_test
y_train = y[y.index <= '1975-12-01']
y_test = y[y.index > '1975-12-01']

In [466]:
y_train.shape, y_test.shape

((270000, 1), (270000, 1))

In [467]:
# Reformating y
y_train = y_train.values.ravel()
y_test = y_test.values.ravel()
y_train.shape, y_test.shape

((270000,), (270000,))

## 2. Models

In [69]:
# Importing packages
import tensorflow as tf
import matplotlib.pyplot as plt

import math
import keras
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import BatchNormalization 
from keras.layers.core import Dense, Dropout
from keras.wrappers.scikit_learn import KerasRegressor
from keras.callbacks import EarlyStopping

from sklearn.metrics import mean_squared_error

from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.metrics import r2_score

### LSTM

In [103]:
from skopt.space import Real, Integer
from skopt import BayesSearchCV

In [237]:
# LSTM model
def create_model(epochs=epochs, batch_size=batch_size, lr=lr, neurons=neurons, dropout=dropout):
    model = Sequential()
    model.add(LSTM(neurons, input_shape=(12, 9), return_sequences=True, stateful=False,  dropout=dropout)) # input_shape: n_timesteps, n_features
    model.add(LSTM(neurons//2, return_sequences=True, stateful=False, dropout=dropout))
    model.add(LSTM(neurons//4, stateful=False))
    model.add(Dense(1))
    optimizer=keras.optimizers.Adam(lr=lr)
    # Compile model
    model.compile(loss='mse', optimizer=optimizer) 
    return model

In [238]:
# Wrapper to run Keras in ScikitLearn
lstm_m = KerasRegressor(build_fn=create_model)

In [283]:
# batch_size=Integer(25, 500)
param_opt = dict(epochs=Integer(50, 200), neurons=Integer(60,108), dropout = Real(0.1, 0.7), lr = Real(1e-5, 1e-3))

In [284]:
# Ensuring data keeps temporal element when split
tscv = TimeSeriesSplit(n_splits=9) 

In [285]:
# Implement early stopping
def on_step(optim_result):
    EarlyStopping(monitor='val_loss', mode='auto', verbose=1, patience=5, min_delta=0.1)
    return True

In [286]:
# Run Bayesian Search
bayesian = BayesSearchCV(
     lstm_m,
     param_opt,
     n_iter=32,
     random_state=500,
     cv=tscv,
     verbose = 0, n_jobs=1
 )

bayesian_result = bayesian.fit(X_train, y_train, callback=on_step)

Epoch 1/130
844/844 [==============================] - 11s 9ms/step - loss: 6.2511e-04
Epoch 2/130
844/844 [==============================] - 7s 9ms/step - loss: 1.3983e-05
Epoch 3/130
844/844 [==============================] - 7s 9ms/step - loss: 1.0373e-05
Epoch 4/130
844/844 [==============================] - 7s 9ms/step - loss: 8.7590e-06
Epoch 5/130
844/844 [==============================] - 7s 9ms/step - loss: 8.2717e-06
Epoch 6/130
844/844 [==============================] - 7s 9ms/step - loss: 7.9929e-06
Epoch 7/130
844/844 [==============================] - 7s 9ms/step - loss: 7.7880e-06
Epoch 8/130
844/844 [==============================] - 7s 9ms/step - loss: 7.6277e-06
Epoch 9/130
844/844 [==============================] - 7s 9ms/step - loss: 7.2741e-06
Epoch 10/130
844/844 [==============================] - 7s 9ms/step - loss: 7.1945e-06
Epoch 11/130
844/844 [==============================] - 7s 9ms/step - loss: 6.8197e-06
Epoch 12/130
844/844 [=============================

844/844 [==============================] - 7s 9ms/step - loss: 4.2702e-06
Epoch 96/130
844/844 [==============================] - 7s 9ms/step - loss: 4.2379e-06
Epoch 97/130
844/844 [==============================] - 7s 9ms/step - loss: 4.1611e-06
Epoch 98/130
844/844 [==============================] - 7s 9ms/step - loss: 4.2541e-06
Epoch 99/130
844/844 [==============================] - 7s 9ms/step - loss: 4.2900e-06
Epoch 100/130
844/844 [==============================] - 7s 9ms/step - loss: 4.3583e-06
Epoch 101/130
844/844 [==============================] - 7s 9ms/step - loss: 4.2172e-06
Epoch 102/130
844/844 [==============================] - 7s 9ms/step - loss: 4.2614e-06
Epoch 103/130
844/844 [==============================] - 7s 9ms/step - loss: 4.1839e-06
Epoch 104/130
844/844 [==============================] - 7s 9ms/step - loss: 4.2313e-06
Epoch 105/130
844/844 [==============================] - 7s 9ms/step - loss: 4.1668e-06
Epoch 106/130
844/844 [===========================

1688/1688 [==============================] - 15s 9ms/step - loss: 3.4557e-06
Epoch 57/130
1688/1688 [==============================] - 15s 9ms/step - loss: 3.2457e-06
Epoch 58/130
1688/1688 [==============================] - 15s 9ms/step - loss: 3.2050e-06
Epoch 59/130
1688/1688 [==============================] - 15s 9ms/step - loss: 3.2951e-06
Epoch 60/130
1688/1688 [==============================] - 15s 9ms/step - loss: 3.3512e-06
Epoch 61/130
1688/1688 [==============================] - 15s 9ms/step - loss: 3.2758e-06
Epoch 62/130
1688/1688 [==============================] - 15s 9ms/step - loss: 3.2602e-06
Epoch 63/130
1688/1688 [==============================] - 15s 9ms/step - loss: 3.3177e-06
Epoch 64/130
1688/1688 [==============================] - 15s 9ms/step - loss: 3.2683e-06
Epoch 65/130
1688/1688 [==============================] - 15s 9ms/step - loss: 3.2624e-06
Epoch 66/130
1688/1688 [==============================] - 15s 9ms/step - loss: 3.2726e-06
Epoch 67/130
1688/1688 

2532/2532 [==============================] - 23s 9ms/step - loss: 3.0796e-06
Epoch 17/130
2532/2532 [==============================] - 23s 9ms/step - loss: 3.0780e-06
Epoch 18/130
2532/2532 [==============================] - 23s 9ms/step - loss: 2.9389e-06
Epoch 19/130
2532/2532 [==============================] - 23s 9ms/step - loss: 2.8754e-06
Epoch 20/130
2532/2532 [==============================] - 23s 9ms/step - loss: 2.8659e-06
Epoch 21/130
2532/2532 [==============================] - 24s 10ms/step - loss: 2.8925e-06
Epoch 22/130
2532/2532 [==============================] - 24s 9ms/step - loss: 2.8760e-06
Epoch 23/130
2532/2532 [==============================] - 23s 9ms/step - loss: 2.8274e-06
Epoch 24/130
2532/2532 [==============================] - 22s 9ms/step - loss: 2.7165e-06
Epoch 25/130
2532/2532 [==============================] - 22s 9ms/step - loss: 2.8056e-06
Epoch 26/130
2532/2532 [==============================] - 22s 9ms/step - loss: 2.6907e-06
Epoch 27/130
2532/2532

2532/2532 [==============================] - 21s 8ms/step - loss: 2.2653e-06
Epoch 108/130
2532/2532 [==============================] - 21s 8ms/step - loss: 2.2536e-06
Epoch 109/130
2532/2532 [==============================] - 21s 8ms/step - loss: 2.1989e-06
Epoch 110/130
2532/2532 [==============================] - 21s 8ms/step - loss: 2.1981e-06
Epoch 111/130
2532/2532 [==============================] - 21s 8ms/step - loss: 2.1694e-06
Epoch 112/130
2532/2532 [==============================] - 21s 8ms/step - loss: 2.2861e-06
Epoch 113/130
2532/2532 [==============================] - 21s 8ms/step - loss: 2.1677e-06: 0s - 
Epoch 114/130
2532/2532 [==============================] - 22s 9ms/step - loss: 2.2228e-06
Epoch 115/130
2532/2532 [==============================] - 21s 8ms/step - loss: 2.1936e-06
Epoch 116/130
2532/2532 [==============================] - 21s 8ms/step - loss: 2.2210e-06
Epoch 117/130
2532/2532 [==============================] - 22s 8ms/step - loss: 2.1872e-06
Epoch 

...

5907/5907 [==============================] - 52s 9ms/step - loss: 1.2206e-06
Epoch 127/130
5907/5907 [==============================] - 52s 9ms/step - loss: 1.2400e-06
Epoch 128/130
5907/5907 [==============================] - 52s 9ms/step - loss: 1.2301e-06
Epoch 129/130
5907/5907 [==============================] - 52s 9ms/step - loss: 1.2343e-06
Epoch 130/130
844/844 [==============================] - 3s 2ms/step - loss: 5.0141e-07
Epoch 1/130
6750/6750 [==============================] - 49s 7ms/step - loss: 5.7720e-05
Epoch 2/130
6750/6750 [==============================] - 46s 7ms/step - loss: 1.7755e-06
Epoch 3/130
6750/6750 [==============================] - 46s 7ms/step - loss: 1.6340e-06
Epoch 4/130
6750/6750 [==============================] - 46s 7ms/step - loss: 1.5893e-06
Epoch 5/130
6750/6750 [==============================] - 46s 7ms/step - loss: 1.5419e-06
Epoch 6/130
6750/6750 [==============================] - 46s 7ms/step - loss: 1.4903e-06
Epoch 7/130
6750/6750 [=====

6750/6750 [==============================] - 46s 7ms/step - loss: 1.2190e-06
Epoch 87/130
6750/6750 [==============================] - 46s 7ms/step - loss: 1.2067e-06
Epoch 88/130
6750/6750 [==============================] - 46s 7ms/step - loss: 1.2040e-06
Epoch 89/130
6750/6750 [==============================] - 46s 7ms/step - loss: 1.1627e-06
Epoch 90/130
6750/6750 [==============================] - 46s 7ms/step - loss: 1.1880e-06
Epoch 91/130
6750/6750 [==============================] - 46s 7ms/step - loss: 1.1849e-06
Epoch 92/130
6750/6750 [==============================] - 46s 7ms/step - loss: 1.1798e-06
Epoch 93/130
6750/6750 [==============================] - 46s 7ms/step - loss: 1.1838e-06
Epoch 94/130
6750/6750 [==============================] - 46s 7ms/step - loss: 1.1817e-06
Epoch 95/130
6750/6750 [==============================] - 46s 7ms/step - loss: 1.1866e-06
Epoch 96/130
6750/6750 [==============================] - 46s 7ms/step - loss: 1.1974e-06
Epoch 97/130
6750/6750 

7594/7594 [==============================] - 67s 9ms/step - loss: 1.2487e-06
Epoch 47/130
7594/7594 [==============================] - 67s 9ms/step - loss: 1.2100e-06
Epoch 48/130
7594/7594 [==============================] - 67s 9ms/step - loss: 1.2014e-06
Epoch 49/130
7594/7594 [==============================] - 67s 9ms/step - loss: 1.2288e-06
Epoch 50/130
7594/7594 [==============================] - 67s 9ms/step - loss: 1.2388e-06
Epoch 51/130
7594/7594 [==============================] - 67s 9ms/step - loss: 1.2015e-06
Epoch 52/130
7594/7594 [==============================] - 67s 9ms/step - loss: 1.2043e-06
Epoch 53/130
7594/7594 [==============================] - 67s 9ms/step - loss: 1.1960e-06
Epoch 54/130
7594/7594 [==============================] - 67s 9ms/step - loss: 1.2058e-06
Epoch 55/130
7594/7594 [==============================] - 67s 9ms/step - loss: 1.2042e-06
Epoch 56/130
7594/7594 [==============================] - 67s 9ms/step - loss: 1.2023e-06
Epoch 57/130
7594/7594 

8438/8438 [==============================] - 76s 9ms/step - loss: 1.4220e-06
Epoch 7/130
8438/8438 [==============================] - 75s 9ms/step - loss: 1.4025e-06
Epoch 8/130
8438/8438 [==============================] - 75s 9ms/step - loss: 1.4097e-06
Epoch 9/130
8438/8438 [==============================] - 75s 9ms/step - loss: 1.3753e-06
Epoch 10/130
8438/8438 [==============================] - 76s 9ms/step - loss: 1.3873e-06
Epoch 11/130
8438/8438 [==============================] - 76s 9ms/step - loss: 1.3593e-06
Epoch 12/130
8438/8438 [==============================] - 76s 9ms/step - loss: 1.3619e-06
Epoch 13/130
8438/8438 [==============================] - 76s 9ms/step - loss: 1.3501e-06
Epoch 14/130
8438/8438 [==============================] - 76s 9ms/step - loss: 1.3221e-06
Epoch 15/130
8438/8438 [==============================] - 75s 9ms/step - loss: 1.3543e-06
Epoch 16/130
8438/8438 [==============================] - 75s 9ms/step - loss: 1.3382e-06
Epoch 17/130
8438/8438 [==

8438/8438 [==============================] - 75s 9ms/step - loss: 1.1402e-06
Epoch 98/130
8438/8438 [==============================] - 75s 9ms/step - loss: 1.1328e-06
Epoch 99/130
8438/8438 [==============================] - 75s 9ms/step - loss: 1.1429e-06
Epoch 100/130
8438/8438 [==============================] - 75s 9ms/step - loss: 1.1280e-06
Epoch 101/130
8438/8438 [==============================] - 75s 9ms/step - loss: 1.1359e-06
Epoch 102/130
8438/8438 [==============================] - 75s 9ms/step - loss: 1.1427e-06
Epoch 103/130
8438/8438 [==============================] - 75s 9ms/step - loss: 1.1106e-06
Epoch 104/130
8438/8438 [==============================] - 75s 9ms/step - loss: 1.1481e-06
Epoch 105/130
8438/8438 [==============================] - 75s 9ms/step - loss: 1.1424e-06
Epoch 106/130
8438/8438 [==============================] - 75s 9ms/step - loss: 1.1230e-06
Epoch 107/130
8438/8438 [==============================] - 75s 9ms/step - loss: 1.1253e-06
Epoch 108/130
8

In [329]:
# Checking parameters
print("Best: %f using %s" % (bayesian_result.best_score_, bayesian_result.best_params_))
means = bayesian_result.cv_results_['mean_test_score']
stds = bayesian_result.cv_results_['std_test_score']
params = bayesian_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: -0.000001 using OrderedDict([('dropout', 0.6886384970038995), ('epochs', 130), ('lr', 0.0006170997584834369), ('neurons', 87)])
-0.000001 (0.000001) with: OrderedDict([('dropout', 0.6886384970038995), ('epochs', 130), ('lr', 0.0006170997584834369), ('neurons', 87)])


In [455]:
# Defining best model parameters
#batch_size = bayesian_result.best_params_['batch_size']
epochs = bayesian_result.best_params_['epochs']
neurons = bayesian_result.best_params_['neurons']
dropout = bayesian_result.best_params_['dropout']
lr = bayesian_result.best_params_['lr']

In [456]:
# Defining optimized model
lstm_opt = Sequential()
lstm_opt.add(LSTM(neurons, input_shape=(12, 9), return_sequences=True, stateful=False,  dropout=dropout)) # input_shape: n_timesteps, n_features
lstm_opt.add(LSTM(neurons//2, return_sequences=True, stateful=False, dropout=dropout))
lstm_opt.add(LSTM(neurons//4, stateful=False))
lstm_opt.add(Dense(1))
optimizer=keras.optimizers.Adam(lr=lr)
# Compile model
lstm_opt.compile(loss='mse', optimizer=optimizer) 

In [447]:
lstm_opt.summary()

Model: "sequential_64"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_114 (LSTM)              (None, 12, 87)            33756     
_________________________________________________________________
lstm_115 (LSTM)              (None, 12, 43)            22532     
_________________________________________________________________
lstm_116 (LSTM)              (None, 21)                5460      
_________________________________________________________________
dense_139 (Dense)            (None, 1)                 22        
Total params: 61,770
Trainable params: 61,770
Non-trainable params: 0
_________________________________________________________________


In [448]:
history = lstm_opt.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, shuffle='batch')

Epoch 1/130
540/540 [==============================] - 26s 42ms/step - loss: 3.2000e-04
Epoch 2/130
540/540 [==============================] - 24s 44ms/step - loss: 4.6741e-06
Epoch 3/130
540/540 [==============================] - 29s 54ms/step - loss: 2.7790e-06
Epoch 4/130
540/540 [==============================] - 29s 54ms/step - loss: 2.1606e-06
Epoch 5/130
540/540 [==============================] - 29s 53ms/step - loss: 1.8877e-06
Epoch 6/130
540/540 [==============================] - 29s 53ms/step - loss: 1.7426e-06
Epoch 7/130
540/540 [==============================] - 29s 53ms/step - loss: 1.6451e-06
Epoch 8/130
540/540 [==============================] - 29s 53ms/step - loss: 1.5684e-06
Epoch 9/130
540/540 [==============================] - 29s 53ms/step - loss: 1.5909e-06
Epoch 10/130
540/540 [==============================] - 29s 53ms/step - loss: 1.4956e-06
Epoch 11/130
540/540 [==============================] - 29s 54ms/step - loss: 1.5274e-06
Epoch 12/130
540/540 [========

540/540 [==============================] - 25s 46ms/step - loss: 1.2907e-06
Epoch 93/130
540/540 [==============================] - 25s 46ms/step - loss: 1.3180e-06
Epoch 94/130
540/540 [==============================] - 25s 46ms/step - loss: 1.3075e-06
Epoch 95/130
540/540 [==============================] - 25s 46ms/step - loss: 1.2883e-06
Epoch 96/130
540/540 [==============================] - 25s 46ms/step - loss: 1.2743e-06
Epoch 97/130
540/540 [==============================] - 25s 46ms/step - loss: 1.2921e-06
Epoch 98/130
540/540 [==============================] - 25s 46ms/step - loss: 1.2908e-06
Epoch 99/130
540/540 [==============================] - 25s 46ms/step - loss: 1.2955e-06
Epoch 100/130
540/540 [==============================] - 25s 46ms/step - loss: 1.3179e-06
Epoch 101/130
540/540 [==============================] - 25s 46ms/step - loss: 1.2880e-06
Epoch 102/130
540/540 [==============================] - 25s 46ms/step - loss: 1.2874e-06
Epoch 103/130
540/540 [========

In [449]:
lstm_predict = lstm_opt.predict(X_test, verbose=0)

In [457]:
r2_df = pd.DataFrame()
r2_df['y'] = y_test
r2_df['y_hat'] = lstm_predict
r2_df['difference'] = (r2_df['y']-r2_df['y_hat'])
r2_df['y_2'] = r2_df['y']**2
r2_df['dif2'] = r2_df['difference']**2
y_2 = r2_df['y_2'].sum() 
dif2 = r2_df['dif2'].sum()
1 - (dif2 / y_2)

0.0009352499605580489

In [458]:
lstm = pd.DataFrame(lstm_predict)

lstm.nunique()

0    260728
dtype: int64

In [459]:
pickle.dump(lstm_predict, open('lstm_predict_6.pkl', 'wb' ))